In [ ]:
pip install -q yahoo_fin

In [ ]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml#egg=runml

In [ ]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

In [ ]:
# Skip some days from consecutive window to lessen test /train overlap
# findata.G_NON_OVERLAP= 0

Run following if like to save results in drive

In [ ]:
drive.mount('/content/drive')
! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print


Mounted at /content/drive
/content/drive/MyDrive/colab/results/etf-11b-adjclose-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
/content/drive/MyDrive/colab/results/etf-11b-high-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras
/content/drive/MyDrive/colab/results/etf-11b-low-sh-1-sc-standard-sbd-0-seq-50-step-20-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-Huber-adam-LSTM-layers-2-units-256.keras


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    # Simplify following using apply
    def calculate_alloc(row):
        gain_low = min(row['Gain_l'], 0)
        gain_high = max(row['Gain_h'], 0)
        gain = min(row['Gain'], gain_high)
        average_gain = (gain_low+gain_high + 2*gain)/4
        if (average_gain == 0):
          return 0
        avg_accuracy = (row['Accu']+row['Accu_l']+row['Accu_h'])/3
        alloc =  avg_accuracy/stop_loss - (1-avg_accuracy)*profit_factor/abs(average_gain)
        return round(max(alloc, 0), 2)
    df['Alloc'] = df.apply(calculate_alloc, axis=1)
    return df

pd.options.display.max_columns = None

In [ ]:
import os
findata.EPOCHS=200
def run_model(prefix, version, tickers, do_train):
  if (not do_train):
    os.system(f"cp  /content/drive/MyDrive/colab/results/{prefix}-{version}-* ./results/")
  mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(),
                         pipeline.AddMA(200), pipeline.Adj()]))
  df= pipeline.runModelCombinedVola(tickers, f"{prefix}-{version}", mod, do_train)
  # Save datafrme to prefix-version-prediction.csv
  df.to_csv(f"results/{prefix}-{version}-prediction.csv")
  return df



In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax' # Change to minmax
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'BYRN', 'CELH', 'CFLT', 'COHR', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ELF', 'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HCP', 'HIMS', 'HOOD', 'HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JAKK', 'JKS', 'KD', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL','MTCH', 'MU',
           'NET', 'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PAR', 'PANW',  'PAYX', 'PD', 'PI', 'PINS',
           'RBLX', 'RKLB', 'RMBS', 'RUM', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'SOUN', 'TEM', 'TEAM', 'TDOC', 'TMDX', 'TNDM', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
df = run_model('ipos', '2a', tickers, True)

In [ ]:
addAllocHL(df, 0.10, 1)
df[(df.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
89,WOLF,2.42,0.89,654.89,1320.74,2025-02-14,2025-03-14,6.18,17.24,1.79,2.03,0.69,462.15,946.46,8.80,0.42,1.47,0.77,410.08,1147.04,7.05,0.14,7.15
83,TTD,2.45,0.94,1014.40,615.86,2025-02-14,2025-03-14,80.16,93.18,0.16,2.02,0.75,1002.11,731.92,104.01,0.30,1.46,0.68,848.13,660.74,87.92,0.10,6.55
85,U,2.60,0.85,409.13,772.98,2025-02-14,2025-03-14,21.51,27.65,0.29,2.25,0.64,203.31,569.88,33.04,0.54,1.50,0.77,412.08,902.35,18.70,-0.13,6.54
75,SOUN,0.38,0.83,139.28,94.62,2025-02-14,2025-03-14,10.96,12.43,0.13,0.31,0.79,232.97,162.01,16.30,0.49,0.16,0.66,50.61,5.55,10.36,-0.05,6.23
64,PUBM,1.00,0.90,200.58,294.84,2025-02-14,2025-03-14,15.70,18.27,0.16,0.86,0.64,71.32,134.41,21.04,0.34,0.51,0.72,156.84,242.57,14.56,-0.07,5.86
31,GTLB,1.93,0.91,930.36,605.88,2025-02-14,2025-03-14,67.83,78.64,0.16,1.19,0.75,881.18,877.20,78.99,0.16,1.05,0.70,712.86,437.40,62.97,-0.07,5.79
92,ZM,8.23,0.74,509.36,700.35,2025-02-14,2025-03-14,85.10,103.92,0.22,5.49,0.59,399.26,408.50,152.71,0.79,3.95,0.71,463.63,947.00,70.45,-0.17,5.59
81,TOST,0.92,0.92,355.57,213.99,2025-02-14,2025-03-14,40.49,46.41,0.15,0.55,0.82,460.56,272.48,47.83,0.18,0.39,0.64,217.38,135.66,34.66,-0.14,5.50
91,Z,2.25,0.92,705.61,473.87,2025-02-14,2025-03-14,79.97,98.39,0.23,2.13,0.71,440.13,354.24,96.65,0.21,1.55,0.60,321.63,144.71,66.70,-0.17,5.20
7,BMBL,1.09,0.83,151.56,244.37,2025-02-14,2025-03-14,8.46,9.30,0.10,0.88,0.66,176.53,236.14,11.81,0.40,0.67,0.73,158.15,236.91,6.63,-0.22,4.66


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'CIBR', 'DAPP', 'DIA', 'DTEC', 'DXYZ',
            'EEM', 'FDN', 'FPX',
            'ICLN', 'IGV', 'IJK', 'IJR', 'IPO',
            'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'JXI', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'URA', 'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
df1 = run_model('etf', '1b', tickers1, True)



In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
49,XME,0.68,0.97,392.56,371.59,60.69,55.83,-0.08,0.44,0.79,350.45,315.67,60.42,-0.00,0.55,0.64,363.95,259.80,53.26,-0.12,9.325000
4,DAPP,0.34,0.96,148.18,182.13,16.87,18.75,0.11,0.26,0.71,158.97,145.53,20.63,0.22,0.25,0.66,130.40,107.76,15.50,-0.08,9.236364
7,DXYZ,2.07,0.93,378.84,74.59,62.91,53.21,-0.15,1.87,0.82,350.60,39.15,62.84,-0.00,1.09,0.55,95.70,-318.62,52.98,-0.16,8.833333
6,DTEC,0.35,0.96,209.30,173.01,47.70,49.59,0.04,0.24,0.71,150.19,153.57,48.12,0.01,0.24,0.64,181.06,135.37,45.94,-0.04,8.600000
20,JETS,0.31,0.96,190.02,154.07,26.25,27.24,0.04,0.16,0.78,145.19,145.35,28.84,0.10,0.21,0.66,102.83,77.59,25.30,-0.04,8.600000
40,XLE,0.98,0.95,605.00,432.48,91.31,87.06,-0.05,0.63,0.83,568.18,413.59,92.96,0.02,0.76,0.68,497.27,264.52,85.88,-0.06,8.500000
41,XLF,0.30,0.96,187.98,158.93,50.85,49.49,-0.03,0.21,0.80,179.19,110.36,49.55,-0.03,0.24,0.63,148.60,80.13,45.98,-0.10,8.266667
48,XLY,1.52,0.95,1148.04,855.93,230.01,220.52,-0.04,1.10,0.73,828.58,645.42,230.46,0.00,1.15,0.66,1017.61,701.53,215.19,-0.06,8.250000
24,MTUM,1.52,0.95,913.42,624.58,223.01,214.26,-0.04,0.80,0.82,805.63,523.72,216.08,-0.03,0.89,0.63,620.82,458.37,203.46,-0.09,8.250000
9,FDN,2.02,0.97,1273.08,791.16,258.50,263.32,0.02,1.37,0.72,993.77,611.73,258.72,0.00,1.41,0.67,1184.25,831.61,245.79,-0.05,8.200000


In [ ]:
df1.to_csv('results/etf_12b_predictions.csv')

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'minmax'
tickers3 = [ 'AIRI', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS',  'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CHGG', 'CLOV', 'CRCT', 'CXM',
            'DLO', 'DM', 'EVGO', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MITK', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PLUG', 'PTON', 'PUBM',
            'REAL', 'REI', 'RIG', 'RVYL',
            'SABR', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SST', 'STEM',
            'TDW', 'TDOC', 'TRUP', 'VTEX', 'VSAT', 'YEXT']
df3 = run_model('vols', '12b', tickers3, False)

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
6,APPS,3.22,0.89,119.07,389.05,2.50,3.22,0.29,3.22,0.62,5.41,273.15,3.30,0.32,1.17,0.68,37.48,364.03,2.18,-0.13,4.855984
37,LUMN,0.57,0.88,46.05,99.02,4.93,6.69,0.36,0.53,0.68,64.04,87.76,6.40,0.30,0.38,0.70,23.08,29.23,4.99,0.01,4.843137
59,SLDP,0.20,0.86,46.25,79.13,1.40,1.92,0.38,0.19,0.60,8.97,43.09,1.83,0.31,0.10,0.73,31.83,68.33,1.37,-0.02,4.690402
46,OPEN,1.51,0.85,52.11,99.35,1.44,2.06,0.43,1.50,0.64,25.98,73.04,2.20,0.53,0.80,0.76,35.35,100.56,1.66,0.16,4.651163
20,DLO,0.76,0.85,153.81,285.68,13.56,19.03,0.40,1.00,0.70,112.43,268.39,22.45,0.66,0.46,0.64,139.60,290.98,12.41,-0.09,4.625000
62,SST,0.33,0.82,60.72,121.44,0.68,1.15,0.69,0.49,0.50,163.68,210.41,1.10,0.62,0.22,0.68,33.06,101.75,0.79,0.16,4.562660
56,RVYL,6.63,0.76,115.83,375.80,1.23,4.90,2.98,6.13,0.54,139.52,278.82,3.67,1.98,1.85,0.66,62.09,232.28,1.84,0.49,4.390051
63,STEM,1.52,0.76,56.41,132.35,0.60,1.89,2.15,1.75,0.56,15.38,86.05,1.20,1.00,0.88,0.65,10.19,68.06,0.56,-0.06,4.358960
11,BKKT,22.91,0.76,447.14,1272.55,18.48,40.71,1.20,38.74,0.47,220.12,994.89,37.53,1.03,7.76,0.74,299.03,1494.75,15.87,-0.14,4.270588
2,ALSMY,0.27,0.88,23.65,23.93,1.96,2.22,0.13,0.29,0.68,10.33,14.72,2.10,0.07,0.24,0.62,9.55,13.06,1.88,-0.04,4.253394


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'KO', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
df4 = run_model('mcap', '1b', tickers4, True)

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
26,META,8.02,0.97,3794.63,1664.59,688.37,610.90,-0.11,6.00,0.78,3836.22,1381.18,688.48,0.00,5.13,0.70,2773.50,735.38,498.78,-0.28,6.655844
4,AMZN,3.10,0.96,1671.51,883.28,233.77,205.71,-0.12,1.74,0.82,1425.03,913.93,234.16,0.00,1.49,0.71,1335.32,1086.15,190.53,-0.18,6.523810
1,ACN,4.67,0.93,2125.91,1720.03,384.83,303.90,-0.21,3.26,0.79,2012.36,1404.43,378.51,-0.02,2.75,0.72,1896.34,1541.58,336.63,-0.13,6.309524
12,COST,11.90,0.91,5137.56,2227.70,978.47,727.57,-0.26,6.99,0.80,4488.53,1642.54,954.29,-0.02,5.77,0.69,4083.76,683.85,827.48,-0.15,6.153846
20,HD,5.05,0.91,2270.81,1404.48,416.38,334.69,-0.20,3.09,0.79,2464.20,1616.96,414.30,-0.01,2.80,0.60,1221.54,429.30,352.79,-0.15,6.050000
18,GOOGL,2.28,0.90,921.36,641.00,199.74,153.12,-0.23,1.80,0.77,1101.88,629.13,198.59,-0.01,1.37,0.62,724.18,480.96,171.00,-0.14,5.993789
19,FDX,4.28,0.92,1677.91,1512.85,265.64,231.55,-0.13,2.98,0.77,2105.18,1793.72,278.14,0.05,2.41,0.72,1902.69,1304.46,229.04,-0.14,5.956044
40,UPS,2.70,0.90,698.42,981.62,111.34,131.96,0.19,1.74,0.76,674.44,1193.59,130.24,0.17,1.58,0.70,668.95,1152.04,121.71,0.09,5.902256
7,BKNG,57.27,0.91,28747.38,14212.10,4758.55,4074.35,-0.14,38.87,0.80,27620.39,14234.91,4895.22,0.03,35.58,0.67,19939.31,7458.83,4108.20,-0.14,5.857143
22,IBM,2.67,0.90,1099.68,625.37,259.31,215.64,-0.17,1.85,0.74,1169.17,445.25,240.17,-0.07,1.56,0.66,914.26,123.03,198.11,-0.24,5.840336


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CX', 'DBOEY', 'DNNGY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FDRVF', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']

df5 = run_model('gstock', '2a', tickers5, False)

In [ ]:
addAllocHL(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
47,PAX,0.17,0.94,62.62,74.70,2025-02-24,2025-03-24,11.76,12.72,0.08,0.11,0.75,80.25,97.29,13.62,0.16,0.10,0.76,89.31,99.92,11.71,-0.00,3.54
13,DNNGY,0.80,0.92,126.48,371.89,2025-02-24,2025-03-24,14.70,15.73,0.07,0.56,0.80,149.24,253.35,17.80,0.21,0.67,0.77,148.96,294.09,13.74,-0.07,3.50
25,FQVLF,0.51,0.91,230.35,315.22,2025-02-24,2025-03-24,12.43,14.28,0.15,0.37,0.73,216.67,306.53,13.69,0.10,0.35,0.65,166.08,272.35,5.35,-0.57,1.95
3,BAK,0.34,0.93,104.31,189.62,2025-02-24,2025-03-24,4.35,4.51,0.04,0.28,0.61,43.78,98.13,5.51,0.27,0.26,0.74,68.69,142.66,4.01,-0.08,1.87
45,ONON,0.89,0.93,421.85,263.76,2025-02-24,2025-03-24,48.75,56.14,0.15,0.55,0.82,591.52,342.64,52.18,0.07,0.67,0.74,281.77,234.53,46.17,-0.05,1.68
49,PROSF,1.64,0.91,678.03,585.15,2025-02-24,2025-03-24,44.55,46.86,0.05,1.24,0.74,448.28,482.18,50.38,0.13,1.24,0.70,328.07,525.94,43.82,-0.02,1.47
27,GLOB,5.16,0.90,1841.17,1918.60,2025-02-24,2025-03-24,153.95,179.59,0.17,3.12,0.75,1877.66,2156.68,164.52,0.07,3.50,0.67,823.61,1078.34,154.61,0.00,1.21
10,CDMGF,0.70,0.69,168.23,329.69,2025-02-21,2025-03-21,22.45,24.95,0.11,0.59,0.55,169.23,170.74,27.32,0.22,0.53,0.57,73.54,270.93,23.40,0.04,0.70
23,FANUY,0.24,0.90,82.18,123.53,2025-02-24,2025-03-24,14.48,14.83,0.02,0.19,0.77,103.66,111.97,16.44,0.14,0.20,0.78,89.29,96.07,12.57,-0.13,0.00
11,CX,0.12,0.95,62.56,50.05,2025-02-24,2025-03-24,6.39,6.76,0.06,0.07,0.80,59.25,59.06,6.90,0.08,0.09,0.59,37.61,20.98,5.88,-0.08,0.00


In [ ]:
findata.G_SCALER = 'standard'
tickers6 = ['ARGT', 'EWD', 'FLN',
           'ECH', 'EPHE', 'EWA', 'EWG', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWT', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'KWT', 'THD', 'TUR', 'VNM', 'XCEM']
df6 = run_model('etf', '12a', tickers6, False)

In [ ]:
addAllocHL(df6, 0.07, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
11,EWP,1.10,0.81,141.60,62.80,2025-02-05,2025-03-05,33.88,27.93,-0.18,0.81,0.70,115.32,57.24,32.03,-0.05,0.85,0.57,54.97,-5.45,27.91,-0.18,7.63
13,EWW,2.76,0.74,158.65,143.62,2025-02-05,2025-03-05,49.53,43.71,-0.12,1.85,0.65,171.98,149.18,49.89,0.01,1.99,0.67,80.52,141.13,43.31,-0.13,6.33
7,EWI,1.52,0.75,124.94,71.43,2025-02-05,2025-03-05,38.92,32.82,-0.16,1.09,0.69,112.33,26.46,37.78,-0.03,0.95,0.53,35.22,-23.00,34.86,-0.10,6.11
22,TUR,1.78,0.75,216.51,110.74,2025-02-05,2025-03-05,35.10,28.75,-0.18,1.21,0.63,181.14,68.51,36.77,0.05,1.27,0.50,71.27,-49.54,28.72,-0.18,5.90
0,ARGT,3.71,0.61,328.57,2.69,2025-02-05,2025-03-05,86.39,69.20,-0.20,2.84,0.64,359.45,-87.61,75.48,-0.13,2.39,0.51,216.35,-98.20,65.19,-0.25,5.84
1,EWD,1.65,0.79,176.28,116.31,2025-02-05,2025-03-05,41.84,36.76,-0.12,1.21,0.61,90.50,73.12,40.03,-0.04,1.18,0.62,81.05,70.48,39.02,-0.07,5.40
8,EWJ,1.99,0.73,201.53,123.89,2025-02-05,2025-03-05,68.84,62.93,-0.09,1.46,0.68,118.48,74.39,68.45,-0.01,1.59,0.60,130.93,91.40,61.41,-0.11,5.02
20,KWT,0.47,0.89,111.70,110.69,2025-02-05,2025-03-05,34.70,32.11,-0.07,0.40,0.63,66.76,51.13,34.76,0.00,0.34,0.66,45.69,65.09,33.05,-0.05,4.63
18,INDA,2.12,0.69,109.06,76.04,2025-02-05,2025-03-05,51.28,45.49,-0.11,1.66,0.57,9.18,-1.12,51.12,-0.00,1.65,0.49,43.38,-9.61,39.83,-0.22,4.55
6,EWG,0.98,0.78,111.54,101.39,2025-02-05,2025-03-05,34.67,32.01,-0.08,0.80,0.60,74.26,16.07,34.15,-0.01,0.83,0.59,44.70,20.30,31.76,-0.08,3.66


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
findata.G_SCALER = 'standard'
tickers7 = ['AAL', 'ADI', 'ALB', 'AMR','ANF', 'APO', 'AQN', 'ARCH', 'ARE', 'ASC',
            'BAH', 'BAX', 'BKR', 'BLK', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IFF', 'IP', 'IPI', 'IVZ',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW', 'ZTS']
df7 = run_model('val', '1a', tickers7, False)

In [ ]:
addAllocHL(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
96,WHR,2.08,0.95,823.72,1056.42,2025-02-12,2025-03-12,98.60,115.64,0.17,1.17,0.76,1021.94,1198.14,118.42,0.20,1.37,0.81,905.34,1398.34,101.92,0.03,7.21
0,AAL,0.44,0.93,153.64,135.54,2025-02-12,2025-03-12,16.44,19.26,0.17,0.29,0.75,155.67,157.07,19.31,0.17,0.27,0.81,177.83,175.70,16.86,0.03,6.97
6,AQN,0.14,0.90,34.86,79.33,2025-02-12,2025-03-12,4.60,5.23,0.14,0.11,0.72,18.76,64.98,6.26,0.36,0.09,0.80,50.94,98.44,4.83,0.05,6.86
13,BGFV,0.30,0.85,43.22,124.77,2025-02-12,2025-03-12,1.44,1.79,0.24,0.23,0.60,36.17,110.71,1.35,-0.07,0.18,0.77,49.91,127.86,-0.68,-1.47,6.69
68,PETS,0.52,0.86,51.19,158.77,2025-02-12,2025-03-12,4.28,6.63,0.55,0.34,0.54,-32.50,116.45,10.32,1.41,0.29,0.73,59.72,158.26,3.58,-0.16,6.61
11,BAX,0.85,0.93,181.11,489.70,2025-02-12,2025-03-12,30.23,32.63,0.08,0.60,0.67,72.92,384.97,39.63,0.31,0.56,0.85,163.04,514.58,29.41,-0.03,6.50
37,F,0.21,0.94,85.38,122.45,2025-02-12,2025-03-12,9.15,10.02,0.09,0.12,0.77,109.67,123.52,11.00,0.20,0.11,0.77,106.90,104.93,9.06,-0.01,6.39
74,RH,8.21,0.92,3598.13,3413.63,2025-02-12,2025-03-12,369.52,407.31,0.10,5.43,0.69,3225.73,3989.27,441.08,0.19,5.79,0.79,3605.49,3624.02,385.49,0.04,5.95
54,KHC,0.34,0.95,155.47,242.23,2025-02-12,2025-03-12,28.50,31.57,0.11,0.22,0.73,135.71,186.18,31.66,0.11,0.22,0.77,161.77,190.18,28.67,0.01,5.94
41,FLO,0.22,0.93,79.01,100.95,2025-02-12,2025-03-12,18.50,19.96,0.08,0.11,0.68,83.57,105.96,22.42,0.21,0.13,0.79,104.40,157.01,19.30,0.04,5.84


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'DBC', 'FXB', 'FXE', 'FXF', 'FXY',
            'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'USO', 'UUP', 'VTIP', 'WEAT']

df8 =  run_model('com', '12a', tickers8, True)

In [ ]:
addAllocHL(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
12,USO,1.56,0.86,427.11,423.33,2025-02-12,2025-03-12,76.65,103.83,0.35,0.94,0.74,525.83,533.82,92.46,0.21,0.85,0.74,601.86,536.13,70.75,-0.08,6.20
15,WEAT,0.12,0.92,39.08,56.30,2025-02-12,2025-03-12,5.08,4.37,-0.14,0.07,0.68,50.93,77.26,5.00,-0.02,0.06,0.83,34.58,71.11,4.66,-0.08,5.99
8,SLV,0.34,0.92,180.11,122.18,2025-02-12,2025-03-12,29.41,26.06,-0.11,0.22,0.79,195.30,151.32,29.26,-0.01,0.20,0.71,151.85,117.42,26.31,-0.11,5.72
5,GLD,1.98,0.88,833.88,408.84,2025-02-12,2025-03-12,268.06,253.92,-0.05,1.13,0.80,1056.80,409.72,261.95,-0.02,1.05,0.64,615.76,148.86,217.45,-0.19,4.61
7,PDBC,0.18,0.89,56.92,57.24,2025-02-12,2025-03-12,13.60,13.27,-0.02,0.10,0.79,67.96,66.11,13.29,-0.02,0.11,0.78,66.61,67.40,12.90,-0.05,0.20
3,FXF,0.38,0.93,183.91,207.75,2025-02-12,2025-03-12,97.38,94.52,-0.03,0.22,0.67,158.60,156.89,97.85,0.00,0.23,0.70,111.44,104.69,95.39,-0.02,0.00
0,DBC,0.26,0.86,66.96,80.78,2025-02-12,2025-03-12,22.38,22.29,-0.00,0.17,0.74,79.04,87.30,22.86,0.02,0.18,0.73,83.42,78.47,21.83,-0.02,0.00
1,FXB,0.54,0.92,238.09,258.73,2025-02-12,2025-03-12,119.85,117.70,-0.02,0.31,0.72,125.82,193.67,122.01,0.02,0.32,0.69,131.26,177.70,118.95,-0.01,0.00
6,ISHG,0.34,0.91,121.41,148.16,2025-02-12,2025-03-12,67.76,65.21,-0.04,0.23,0.67,62.26,113.73,67.57,-0.00,0.24,0.73,79.38,132.66,66.09,-0.02,0.00
4,FXY,0.67,0.89,134.89,228.16,2025-02-12,2025-03-12,59.76,60.85,0.02,0.43,0.63,58.88,164.87,60.61,0.01,0.39,0.74,49.97,212.73,59.26,-0.01,0.00


In [ ]:
findata.EPOCHS=200
findata.G_SCALER = 'standard'
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'NEAR-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
df9 = run_model('cry', '11b', tickers9, False)

In [ ]:
def show_prediction(name, suffix):
  df = pd.read_csv(f'results/{name}_{suffix}_predictions.csv')
  df['predicted_return'] = ((df['predicted_max']+df['predicted_min']+df['predicted_close'])/3 - df['current_price'])/df['current_price']
  return df.sort_values('predicted_return', ascending=False).head(50)


In [ ]:
addAllocHL(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Error,Accu,Buy,Sell,CurrentDate,PredictionDate,Last,Pred,Gain,Error_h,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Error_l,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Alloc
14,TON-USD,1.17,0.68,9.76,37.88,2025-02-14,2025-03-14,0.05,2.15,43.39,1.96,0.52,41.41,59.50,5.27,107.65,0.85,0.61,-10.56,23.41,0.20,3.03,4.30
13,SOL-USD,19.03,0.74,1905.95,1368.12,2025-02-14,2025-03-14,200.21,123.62,-0.38,17.85,0.58,917.03,284.86,199.46,-0.00,12.74,0.66,1642.83,624.63,115.58,-0.42,3.56
8,ICP-USD,8.08,0.61,65.02,148.73,2025-02-14,2025-03-14,7.30,17.91,1.45,8.97,0.36,71.03,-506.35,51.80,6.10,4.15,0.53,-2.00,105.14,7.75,0.06,3.35
3,BCH-USD,101.36,0.58,2651.77,3327.93,2025-02-14,2025-03-14,343.44,550.53,0.60,91.87,0.52,665.11,677.44,613.85,0.79,53.31,0.62,1218.83,1846.78,343.59,0.00,3.24
7,FIL-USD,10.32,0.62,24.31,141.62,2025-02-14,2025-03-14,3.55,-0.75,-1.21,11.67,0.41,-41.56,-56.87,25.67,6.23,8.02,0.54,-0.78,111.55,-0.17,-1.05,3.05
0,ADA-USD,0.21,0.70,4.97,8.06,2025-02-14,2025-03-14,0.81,1.09,0.35,0.16,0.51,0.32,0.82,1.32,0.64,0.14,0.59,1.81,0.40,0.71,-0.12,2.97
2,BTC-USD,6010.88,0.74,693972.68,273881.67,2025-02-14,2025-03-14,97428.95,67363.75,-0.31,4703.86,0.58,422635.57,123708.00,95082.63,-0.02,4126.37,0.57,298861.34,94304.73,63719.50,-0.35,2.97
9,LINK-USD,3.63,0.64,115.96,97.54,2025-02-14,2025-03-14,19.59,25.40,0.30,4.61,0.55,21.91,15.92,28.60,0.46,2.31,0.59,81.64,15.04,17.67,-0.10,2.54
16,XRP-USD,0.24,0.56,6.26,-2.30,2025-02-14,2025-03-14,2.73,1.54,-0.44,0.26,0.44,-0.61,-15.63,2.64,-0.03,0.13,0.59,1.33,-0.77,1.56,-0.43,2.35
11,MATIC-USD,0.24,0.63,3.38,11.03,2025-02-14,2025-03-14,0.33,0.58,0.79,0.20,0.47,-4.34,3.51,0.43,0.32,0.11,0.62,2.00,11.48,0.34,0.06,2.32


In [ ]:

df = show_prediction('etf', '0202')
df

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
38,XLB,88.79,2025-01-31,2025-02-28,88.13,101.84,94.51,0.067988
10,FPX,131.42,2025-01-31,2025-02-28,130.68,147.53,139.43,0.059301
47,XLV,146.87,2025-01-31,2025-02-28,144.84,159.36,158.29,0.049658
13,IPO,46.92,2025-01-31,2025-02-28,46.14,50.67,50.26,0.044828
4,DAPP,15.18,2025-01-31,2025-02-28,14.62,17.00,15.92,0.043917
31,URA,28.31,2025-01-31,2025-02-28,27.68,31.26,29.70,0.043683
32,VNQ,90.55,2025-01-31,2025-02-28,89.14,95.40,97.48,0.038174
11,ICLN,11.37,2025-01-31,2025-02-28,11.16,12.33,11.92,0.038112
30,TDIV,80.75,2025-01-31,2025-02-28,80.02,88.33,82.90,0.037152
28,SMOG,99.15,2025-01-31,2025-02-28,96.63,106.53,105.05,0.036174


In [ ]:
show_prediction('pennystock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
1,AEXAF,0.00,2025-02-20,2025-03-20,0.00,0.00,0.01,inf
64,TDW,50.21,2025-02-21,2025-03-21,47.10,70.12,92.32,0.391091
14,BLNK,1.01,2025-02-21,2025-03-21,1.01,1.50,1.70,0.389439
26,GRPN,12.81,2025-02-21,2025-03-21,12.22,19.43,21.18,0.374707
63,STEM,0.58,2025-02-21,2025-03-21,0.54,0.97,0.88,0.373563
46,OPEN,1.38,2025-02-21,2025-03-21,1.11,2.65,1.73,0.326087
21,DM,2.35,2025-02-21,2025-03-21,2.01,4.11,3.05,0.300709
36,LFMD,5.97,2025-02-21,2025-03-21,5.24,8.49,9.18,0.279174
35,LAZR,6.45,2025-02-21,2025-03-21,5.76,9.06,9.79,0.271835
55,RIG,3.37,2025-02-21,2025-03-21,2.97,5.62,3.85,0.230465


In [ ]:
show_prediction('bluechipstock', '0221').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
39,UNH,469.15,2025-02-21,2025-03-21,461.60,523.46,472.49,0.035596
24,KO,71.58,2025-02-21,2025-03-21,69.67,78.12,74.51,0.035205
19,FDX,253.41,2025-02-21,2025-03-21,249.44,279.59,255.99,0.032609
11,CRM,309.11,2025-02-21,2025-03-21,298.40,330.09,325.45,0.028695
29,NKE,75.92,2025-02-21,2025-03-21,73.20,79.00,81.42,0.025729
21,INTC,24.97,2025-02-21,2025-03-21,24.39,26.38,25.87,0.023094
34,QCOM,166.56,2025-02-21,2025-03-21,162.67,174.01,174.04,0.022094
17,DIS,108.68,2025-02-21,2025-03-21,106.01,119.91,107.23,0.021807
0,AAPL,247.84,2025-02-21,2025-03-21,240.32,263.57,254.77,0.020363
33,PYPL,75.25,2025-02-21,2025-03-21,72.19,82.18,75.21,0.016966


In [ ]:
show_prediction('country', '0224').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
18,INDA,49.26,2025-02-24,2025-03-24,47.76,52.16,52.25,0.029706
7,EWI,40.79,2025-02-24,2025-03-24,40.53,42.72,41.35,0.018223
2,FLN,16.74,2025-02-24,2025-03-24,16.04,17.60,17.24,0.013142
6,EWG,36.07,2025-02-24,2025-03-24,35.65,37.27,36.66,0.012661
3,ECH,28.88,2025-02-24,2025-03-24,27.68,30.05,29.80,0.010272
24,XCEM,30.22,2025-02-24,2025-03-24,28.93,31.71,30.95,0.010258
12,EWT,52.57,2025-02-24,2025-03-24,51.29,54.14,53.88,0.010145
21,THD,55.65,2025-02-24,2025-03-24,54.61,58.58,55.23,0.008805
9,EWC,41.38,2025-02-24,2025-03-24,40.41,42.67,42.14,0.008700
14,EWY,57.47,2025-02-24,2025-03-24,53.80,60.64,59.28,0.007598


In [ ]:
show_prediction('global', '0225').head(10)

,ticker,current_price,current_date,prediction_date,predicted_min,predicted_max,predicted_close,predicted_return
13,DNNGY,14.70,2025-02-24,2025-03-24,14.36,17.30,16.15,0.084127
34,MELI,2223.35,2025-02-24,2025-03-24,2154.47,2500.53,2352.07,0.050527
58,TEF,4.38,2025-02-24,2025-03-24,4.26,4.96,4.56,0.048706
43,NVO,90.59,2025-02-24,2025-03-24,89.25,101.49,93.75,0.046804
15,EIFZF,36.70,2025-02-21,2025-03-21,35.22,41.76,37.60,0.040690
65,VALE,9.97,2025-02-24,2025-03-24,9.69,10.72,10.63,0.037780
25,FQVLF,12.43,2025-02-24,2025-03-24,11.73,14.06,12.84,0.035935
18,EQNR,23.35,2025-02-24,2025-03-24,22.75,25.00,24.70,0.034261
4,BASFY,12.76,2025-02-24,2025-03-24,12.53,13.37,13.65,0.033177
22,EVKIY,9.93,2025-02-21,2025-03-21,9.59,10.38,10.70,0.029540


In [ ]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
country_0224_all_trades.csv
country_0224_close_model.pth
country_0224_max_model.pth
country_0224_min_model.pth
country_0224_predictions.csv
country_0225_all_trades.csv
global_0225_all_trades.csv
global_0225_close_model.pth
global_0225_max_model.pth
global_0225_min_model.pth
global_0225_predictions.csv

sent 21,654,698 bytes  received 225 bytes  43,309,846.00 bytes/sec
total size is 50,533,229  speedup is 2.33
